Linear regression model that predicts the amount of views a YouTube video will get

In the realm of AI/ML engineering, as valuable it is to be able to curate your own dataset, it is an equally important skill to be able to find (usually publically available, but essentially any that aren't created by you) datasets that provide high-quality, clean data that is large enough in amounts to ensure your model has the data it needs to be able to churn out highly-accurate outputs.

For this YouTube regression model, we are going to use a publically available dataset from a popular source, Kaggle.

In [ ]:
# first we need to install the kaggle package
%pip install kaggle

: 

In [ ]:
# then we need to set up the Kaggle API credentials
import os
os.environ["KAGGLE_USERNAME"] = "your_username"
os.environ["KAGGLE_KEY"] = "your_key"

In [ ]:
# then we use the Kaggle API to download the dataset
%kaggle datasets download -d rsrishav/youtube-trending-video-dataset
%unzip youtube-new.zip -d data/

If any of the above does not work for you, don't worry. This repository comes with all datasets downloaded with it, so you can skip the Kaggle stage in case its taking too much your time and move on to the rest of this notebook.